# Text Preprocessing and Annotation Notebook
This notebook contains all the code I used for preprocessing, annotating, and saving the a corpus of all Pokémon and their descriptions. 

## 1. Getting my data
   
```
import requests
import pandas as pd
import os
import time

os.makedirs("data", exist_ok=True)
pokemon_data = []

for i in range(1, 1026):
    print(f"Getting Pokémon {i}...") #just to see what the downloading status is 
    
    for attempt in range(3):
        p_resp = requests.get(f"https://pokeapi.co/api/v2/pokemon/{i}")
        if p_resp.status_code == 200:
            break
        print(f"Retry {attempt+1} for Pokémon {i} (status {p_resp.status_code})")
        time.sleep(1)
    else:
        print(f"Skipping Pokémon {i} after 3 failed attempts")     #retry to get info up to 3 times
        continue

    p = p_resp.json()

    for attempt in range(3):
        s_resp = requests.get(f"https://pokeapi.co/api/v2/pokemon-species/{i}")
        if s_resp.status_code == 200:
            break
        print(f"Retry {attempt+1} for species {i} (status {s_resp.status_code})")
        time.sleep(1)
    else:
        print(f"Skipping species {i} after 3 failed attempts")
        continue

    s = s_resp.json()

    english_entries = [
        e['flavor_text'].replace("\n", " ").replace("\x0c", " ")
        for e in s['flavor_text_entries']
        if e['language']['name'] == 'en'
    ]
    description = english_entries[0] if english_entries else ""

    filename = f"{i:04d}_{p['name']}.txt"
    with open(os.path.join("data", filename), "w", encoding="utf-8") as f:
        f.write(description)

    pokemon_data.append({
        "id": i,
        "name": p['name'],
        "types": [t['type']['name'] for t in p['types']],
        "description": description,
        "filename": filename
    })

    time.sleep(0.3)  # delay to avoid overload

df = pd.DataFrame(pokemon_data)
df.to_csv("pokedex.csv", index=False)
print("Done! CSV and .txt files saved in 'data/' folder.")

```

## 2. Imports and preprocessing 

```
>>> import spacy
>>> nlp = spacy.load("en_core_web_sm")
>>> import os
>>> from spacy import displacy
>>> from IPython.display import display, HTML
>>> import spacy
>>> import pandas as pd
>>> pd.options.mode.chained_assignment = None
>>> import plotly.express as px
>>> texts = []
>>> file_names = []

>>> for _file_name in os.listdir('pokedata'):
...     if _file_name.endswith('.txt'):
...             texts.append(open('pokedata' + '/' + _file_name, 'r', encoding='utf-8').read())
...             file_names.append(_file_name)
... 

>>> metadata_df = pd.read_csv('pokemetadata.csv', delimiter=';')
>>> final_poke_df = metadata_df.merge(pokedata_df,on='filename')

>>> def preprocess_text(text):
...     text = text.lower()
...     text = re.sub(r'http\S+|www\S+', '', text)
...     text = re.sub(r'\S+@\S+', '', text)
...     text = re.sub(r'\d{2,4}[-.\s]?\d{3,4}[-.\s]?\d{3,4}', '', text)
...     text = re.sub(r'[^\w\s]', '', text)
...     text = re.sub(r'\s+', ' ', text).strip()
...     return text
... 

>>> final_poke_df['clean_text'] = final_poke_df['text'].apply(preprocess_text)

```

## 3. Adding Annotations

```
>>> nlp = spacy.load('en_core_web_sm')
>>> print(nlp.pipe_names)
['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

>>> def process_text(text):
...      return nlp(text)
... 
>>> final_poke_df['doc'] = final_poke_df['clean_text'].apply(process_text)

>>> def get_token(doc):
...      return [(token.text) for token in doc]
... 
>>> final_poke_df['tokens'] = final_poke_df['doc'].apply(get_token)
>>> def get_lemma(doc):
...     return [(token.lemma_) for token in doc]
... 
>>> final_poke_df['lemmas'] = final_poke_df['doc'].apply(get_lemma)
>>> def get_pos(doc):
...     return [(token.pos_, token.tag_) for token in doc]
... 
>>> final_poke_df['POS'] = final_poke_df['doc'].apply(get_pos)
>>> def extract_proper_nouns(doc):
...     return [token.text for token in doc if token.pos_ == 'PROPN']
... 
>>> final_poke_df['proper_nouns'] = final_poke_df['doc'].apply(extract_proper_nouns)
>>> final_poke_df = final_poke_df.drop(['clean_text'], axis=1)
>>> final_poke_df.to_csv('pokedata_preprocessed_annotated.csv')
>>> exit()
```